In [1]:
! pip install pandas matplotlib seaborn yfinance

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.0 MB 1.0 MB/s eta 0:00:03
     ---------- ----------------------------- 0.8/3.0 MB 1.2 MB/s eta 0:00:02
     ------------- -------------------------- 1.0/3.0 MB 1.2 MB/s eta 0:00:02
     ----------------- ---------------------- 1.3/3.0 MB 1.2 MB/s eta 0:00:02
     -------------------- ------------------- 1.6/3.0 MB 1.2 MB/s eta 0:00:02
     ------------------------ --------------- 1.8/3.0 MB 1